In [6]:
import os

import numpy as np

from lexnorm.data import lexicon
from lexnorm.data import norm_dict
from lexnorm.data.normEval import loadNormData
from lexnorm.definitions import DATA_PATH

In [33]:
raw, norm = loadNormData(os.path.join(DATA_PATH, "interim/train.txt"))
norm_dictionary = norm_dict.construct(os.path.join(DATA_PATH, "interim/train.txt"))
norm_dictionary.keys()

dict_keys(['ah', 'why', 'sub', 'ozil', 'opolo', 'eyes', 'u', 'no', 'fit', 'open', 'have', 'a', 'very', 'sexy', 'header', 'rawr', 'i', 'miss', 'my', 'bie', 'where', 'wanna', 'out', 'wif', 'me', 'wonderful', 'day', 'like', 'swan', 'haha', 'cantik', 'julie', 'christie', '1968', 'photograph', 'by', 'richard', 'avedon', 'did', 'calum', 'slip', 'omfg', 'and', 'you', 'didnt', 'make', 'this', 'brah', 'friend', 'jack', 'storm', 'pshh', 'went', 'got', 'mine', 'done', 'regardless', 'im', 'sick', 'of', 'not', 'being', 'able', 'to', 'be', 'girl', 'syempre', 'in', 'the', 'right', 'age', 'hahaha', 'shit', 'rockin', 'go', 'head', 'show', 'that', 'bria', 'words', 'describe', 'exo', 'luhan', 'is', 'pain', 'chanyeol', 'sex', 'sehun', 'porn', 'ukip', 'has', 'had', 'two', 'councillors', 'elected', 'northern', 'ireland', 'today', 'marvellous', 'lol', 'cause', 'said', 'woulda', 'been', 'falling', 'along', 'with', 'yea', 'was', 'gonna', 'hit', 'up', 'but', 'guess', 'okay', 'well', 'at', 'least', 'know', 'othe

In [8]:
lex = lexicon.build(
        {"english", "american"},
        {"contractions", "proper-names", "upper", "words"},
        50,
        1,
    )

lex = lexicon.refine(lex.union(lexicon.build_abbreviations()))

In [9]:
lexicon.evaluate(norm_dictionary, lex)

Correlation of in lexicon with normalisation: 0.43
Most common normalised raw phrases in lexicon: [(('nigga', 'nigger'), 57), (('niggas', 'niggers'), 52), (('ur', 'your'), 33), (('gonna', 'going to'), 29), (('bout', 'about'), 21), (('tho', 'though'), 17), (('wit', 'with'), 17), (('cause', 'because'), 16), (('wanna', 'want to'), 12), (('ur', "you're"), 12), (('cant', "can't"), 12), (('nd', 'and'), 11), (('yo', 'you'), 11), (('ill', "i'll"), 9), (('dis', 'this'), 9), (('yo', 'your'), 8), (('em', 'them'), 8), (('yea', 'yeah'), 7), (('its', "it's"), 7), (('rn', 'right now'), 6)]
Most common un-normalised raw phrases not in lexicon: [(('haha', 'haha'), 81), (('bae', 'bae'), 45), (('x', 'x'), 44), (('2', '2'), 40), (('niall', 'niall'), 40), (('hahaha', 'hahaha'), 36), (('idk', 'idk'), 36), (('1', '1'), 34), (('rp', 'rp'), 33), (('exo', 'exo'), 31), (('wtf', 'wtf'), 26), (('t', 't'), 25), (('5', '5'), 25), (('zayn', 'zayn'), 23), (('2014', '2014'), 23), (('c', 'c'), 21), (('smh', 'smh'), 21),

In [10]:
def original_token(tok):
    # MONOISE
    # needed if detect step is skipped, as all tokens will be replaced by one from the list of candidates
    return {tok}

In [11]:
import gensim
w2v_vectors = gensim.models.KeyedVectors.load_word2vec_format(
        os.path.join(DATA_PATH, "external/monoise_data/w2v_100.bin"),
        binary=True,
        unicode_errors="ignore",
    )
# load embeddings from van der goot. Used params -size 100 -window 5 -cbow 0 -binary 1 -threads 45
# there are 400 length embeddings which supposedly give slight performance improvement, but too slow
# unicode incompatibilities present so must ignore

In [12]:
train_model = model = gensim.models.Word2Vec(sentences=raw, vector_size=100, window=5)
# get keyed vectors only as finished training model
train_vectors = train_model.wv
w2v_vectors.add_vectors(train_vectors.index_to_key, train_vectors.vectors)

In [13]:
from lexnorm.models.filtering import is_eligible


def word_embeddings(tok, vectors, threshold=0):
    # TODO uni, bigram freqs?
    # TODO implement word2vec with keras. Use newer embeddings. Experiment with different no. of candidates generated. Could even create twitter embeddings myself? Could clean up as VDG did before creating train embeddings. Cosine similarity threshold?
    # MONOISE
    # can use twitter embeddings from van der Goot - based on distributional hypothesis to find tokens with similar semantics
    # could use cosine similarity as a feature for selection? Using here to get most similar candidates.
    # ISSUE: antonyms also often present in same contexts.
    candidates = set()
    if tok in vectors:
        candidates = set(vectors.similar_by_vector(tok))
    return {c[0].lower() for c in candidates if is_eligible(c[0]) and c[1] >= threshold}

# pretty much no tokens in train set not in twitter embeddings from vdg but can't assume this is the case
# possible that tok will not be in vectors
w2v_vectors.similar_by_vector("suree")
# pretty terrible performance for train_vectors - to be expected as such a low amount of data could literally just use external set as contains almost every word anyway and will be much more accurate

/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


[('sureee', 0.906640350818634),
 ('suure', 0.8680776357650757),
 ('sureeee', 0.8335921168327332),
 ('suree,', 0.8297243118286133),
 ('sureee,', 0.8246626257896423),
 ('okkay', 0.8242574334144592),
 ('ohkayy', 0.8241530060768127),
 ('surre', 0.8228244185447693),
 ('alrighht', 0.8206066489219666),
 ('suree!', 0.817251980304718)]

In [14]:
def lookup(tok, dictionary):
    # TODO: external norm dicts?
    # MONOISE
    # lookup in list of all replacement pairs found in the training data (and external sources?)
    # all norm tokens with raw token tok are included as candidates
    return [(k, v) for k, v in dictionary.get(tok, {}).items()]

lookup("bitch", norm_dictionary)
# norm_dictionary

[('bitch', 28)]

In [15]:
def clipping(tok, lex):
    # MONOISE
    # all words in lexicon that have tok as a prefix (capturing abbreviation). May only consider for tok length above 2?
    candidates = set()
    if len(tok) < 2:
        return set()
    # TODO: length threshold? prune generated (only some degree of clipping allowed w.r.t. edit distance)?
    return [t for t in lex if t.startswith(tok)]

In [16]:
# TODO: number of candidates on average generated by each module and with all modules
# TODO: which modules contribute the most / most unique correct candidates
# TODO: for modules and whole, percentage of correct vs incorrect candidates

In [17]:
def split(tok, lex):
    # MONOISE
    # hypothesis splits on (every/some) position and check if both words are in lexicon. May only consider of tok length above 3?
    candidates = set()
    if len(tok) < 3:
        return set()
    for pos in range(1, len(tok)):
        left = tok[:pos]
        right = tok[pos:]
        if left in lex and right in lex:
            candidates.add(" ".join([left, right]))
    # TODO: recursive candidate generation on each left and right? Probably not... More than one split? Probably not either...
    # TODO: length threshold?
    return candidates

In [18]:
from spylls.hunspell import Dictionary

def spellcheck(tok):
    # TODO: no control over this - can I change in source code? Try and load in custom lexicon.
    dictionary = Dictionary.from_files('en_US')
    return {c.lower() for c in dictionary.suggest(tok)}

In [19]:
# TODO: contextual features?
# def generate_candidates(tweet):
#     for token in tweet:
#         ...

In [20]:
def generate_candidates(tok):
    candidates = set()
    candidates = candidates.union(original_token(tok))
    candidates = candidates.union(word_embeddings(tok, w2v_vectors))
    candidates = candidates.union(spellcheck(tok))
    # obviously lookup on the train set will always give the correct answer!
    # candidates = candidates.union(lookup(tok, norm_dictionary))
    candidates = candidates.union(clipping(tok, lex))
    candidates = candidates.union(split(tok, lex))
    return candidates

In [21]:
from lexnorm.evaluation import condition_normalisation
from lexnorm.data import normEval
import os
from lexnorm.definitions import DATA_PATH
from lexnorm.data import norm_dict
from lexnorm.models.filtering import is_eligible
from random import sample

norm_dictionary = norm_dict.construct(os.path.join(DATA_PATH, "interim/train.txt"))
# keys = sample(list(norm_dictionary.keys()), 100)
# assert condition_normalisation.contingency(
#     raw, norm, lambda x: x[0] == "a", True
# ) == condition_normalisation.contingency_from_dict(
#     norm_dictionary, lambda x: x[0][0] == "a"
# )
a, b, c, d = condition_normalisation.contingency_from_dict(
    norm_dictionary, lambda x: not x[1] in generate_candidates(x[0])
)

# print(sum(a.values()) + sum(b.values()) + sum(c.values()) + sum(d.values()))

# a2, b2, c2, d2 = condition_normalisation.contingency(raw, norm, lambda x: x[0] == "a", True)

# print(sum(a.values()) + sum(b.values()) + sum(c.values()) + sum(d.values()))

KeyboardInterrupt: 

In [22]:
# TODO merge module that checks some tokens ahead of current token (perhaps only one)

In [23]:
import pandas as pd
candidates = pd.DataFrame(columns=["feature1", "feature2", "feature3"])
candidates.loc["testing"] = [1, 3]
candidates
# new_candidates = pd.DataFrame(columns=)

ValueError: cannot set a row with mismatched columns

In [39]:
import lexnorm.models.candidate_generation as candidate_generation
import importlib
import numpy as np
from spylls.hunspell import Dictionary
importlib.reload(candidate_generation)
dictionary = Dictionary.from_files("en_US")

In [42]:
train_data = pd.DataFrame()
candidates = candidate_generation.candidates_from_token("bruh", w2v_vectors, norm_dictionary, lex, dictionary)
# candidates.cosine_to_orig = candidates.index.map(lambda x: w2v_vectors.similarity(x, "lol") if x in w2v_vectors else 0)
# w2v_vectors.similarity(candidates.index, "lol") if indexes in w2v_vectors else 0
# candidates.index.to_series()
candidates

NameError: name 'candidate_generation' is not defined

In [ ]:
d1 = pd.DataFrame(columns=["feature"])
d2 = pd.DataFrame(columns=["feature"])
d1.loc["key"] = {"feature": np.nan}
d2.loc["key"] = {"feature": 1}
d2.loc["key2"] = {"feature": 3}

In [28]:
d1.combine_first(d2)

NameError: name 'd1' is not defined

In [74]:
df = pd.read_csv(os.path.join(DATA_PATH, "interim/candidates.txt"), index_col=0)

In [76]:
# df.set_index(["raw_tok_index", "candidate"], verify_integrity=True)
# df.groupby(["raw_tok_index"]).sum()
# df[np.isnan(df["norms_seen"])].groupby("raw_tok_index").sum()
df[df["raw_tok_index"] == "0_1"]

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length,correct,raw_tok_index
bet,0.458454,NaN,NaN,NaN,NaN,NaN,15.0,1.0,3,NaN,60.0,1.0,1.0,3.0,NaN,0_1
find,0.681112,5.0,NaN,NaN,NaN,NaN,NaN,1.0,4,NaN,60.0,1.0,1.0,3.0,NaN,0_1
gee,0.208263,NaN,NaN,NaN,NaN,NaN,3.0,1.0,3,NaN,60.0,1.0,1.0,3.0,NaN,0_1
gel,0.280316,NaN,NaN,NaN,NaN,NaN,9.0,1.0,3,NaN,60.0,1.0,1.0,3.0,NaN,0_1
gem,0.306635,NaN,NaN,NaN,NaN,NaN,12.0,1.0,3,NaN,60.0,1.0,1.0,3.0,NaN,0_1
gen,0.236195,NaN,NaN,NaN,NaN,NaN,6.0,1.0,3,NaN,60.0,1.0,1.0,3.0,NaN,0_1
gent,0.192144,NaN,NaN,NaN,NaN,NaN,2.0,1.0,4,1.0,60.0,1.0,1.0,3.0,NaN,0_1
get,1.000000,NaN,1.0,1.0,NaN,60.0,16.0,1.0,3,1.0,60.0,1.0,1.0,3.0,1.0,0_1
getaway,0.245932,NaN,1.0,NaN,NaN,NaN,NaN,1.0,7,1.0,60.0,1.0,1.0,3.0,NaN,0_1
getaway's,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,9,1.0,60.0,1.0,1.0,3.0,NaN,0_1


In [41]:
candidate_generation.candidates_from_tweets(raw[:2], w2v_vectors, norm_dictionary, lex, dictionary)

/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packages/gensim/models/keyedvectors.py:849: RuntimeWarning: invalid value encountered in divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]
/Users/elijoe/Library/Mobile Documents/com~apple~CloudDocs/Documents/2/Diss/lexnorm/.venv/lib/python3.10/site-packa

,cosine_to_orig,embeddings_rank,from_clipping,from_original_token,from_split,norms_seen,spellcheck_rank,in_lexicon,length,same_order,orig_norms_seen,orig_in_lexicon,orig_same_order,orig_length
a,0.406318,NaN,NaN,NaN,NaN,NaN,3.0,1,1,0,8.0,1.0,1.0,2.0
aah,0.734173,NaN,NaN,NaN,NaN,NaN,5.0,0,3,1,8.0,1.0,1.0,2.0
agu,0.764286,1.0,NaN,NaN,NaN,NaN,NaN,0,3,0,8.0,1.0,1.0,2.0
ah,1.000000,NaN,1.0,1.0,NaN,8.0,NaN,1,2,1,8.0,1.0,1.0,2.0
aha,0.422756,NaN,1.0,NaN,NaN,NaN,6.0,1,3,1,8.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rawer,0.301921,NaN,NaN,NaN,NaN,NaN,1.0,1,5,1,2.0,0.0,1.0,4.0
rawr,1.000000,NaN,NaN,1.0,NaN,2.0,NaN,0,4,1,2.0,0.0,1.0,4.0
rawrr,0.781826,0.0,NaN,NaN,NaN,NaN,NaN,0,5,1,2.0,0.0,1.0,4.0
rawrrr,0.772067,1.0,NaN,NaN,NaN,NaN,NaN,0,6,1,2.0,0.0,1.0,4.0


In [161]:
df = pd.read_csv(os.path.join(DATA_PATH, "../hpc/candidates.txt"), index_col=0)

In [167]:
# get all tokens where correct normalisation not produced by candidate generation
filtered = df.groupby("raw_tok_index").filter(lambda x: x.sum()["correct"] == 0)
ungenerated = filtered.loc[filtered["from_original_token"] == 1.0]["gold"]

In [95]:
raw, _ = loadNormData(os.path.join(DATA_PATH, "raw/dev.norm"))

In [96]:
count = 0
for raw_tweet, norm_tweet in zip(raw, norm):
    for raw_tok, norm_tok in zip(raw_tweet, norm_tweet):
        if is_eligible(raw_tok):
            count += 1
count
# as expected - perhaps can be used to test candidate_generation

6876

In [174]:
from collections import Counter

Counter(zip(ungenerated.index, ungenerated)).most_common()
# very few ungenerated correct candidates! So candidate generation module perhaps alright.

[(('v', 'very'), 2),
 (('hapi', 'happy'), 2),
 (('witchu', 'with you'), 2),
 (('yessss', 'yes'), 1),
 (('bestie', 'best friend'), 1),
 (('niggra', 'nigger'), 1),
 (('wada', 'water'), 1),
 (('chu', 'you'), 1),
 (('nows', 'now is'), 1),
 (('nuh', 'know'), 1),
 (('ntn', 'nothing'), 1),
 (('mnl', 'my new love'), 1),
 (('za', 'that'), 1),
 (('skepta', 'sunglasses'), 1),
 (('whatdoiwear', 'what do i wear'), 1),
 (('brutha', 'brother'), 1),
 (('yah', 'you'), 1),
 (('nuff', 'enough'), 1),
 (('shizz', 'shit'), 1),
 (('nuffin', 'nothing'), 1),
 (('diss', 'this'), 1),
 (('vas', 'was'), 1),
 (('redsox', 'red sox'), 1),
 (('nem', 'they'), 1),
 (('fkn', 'fucking'), 1),
 (('sim', 'seems'), 1),
 (('hbu', 'how about you'), 1),
 (('goddamit', 'god damn it'), 1),
 (('satnite', 'saturday night'), 1),
 (('dese', 'these'), 1),
 (('summn', 'something'), 1),
 (('cums', 'comes'), 1),
 (('dnt', "doesn't"), 1),
 (('getcha', 'get you'), 1),
 (('trynna', 'trying to'), 1),
 (('ya', 'your'), 1),
 (('lottle', 'lot'),

In [183]:
[x for x in dictionary.suggest("brotha")]

['broths',
 'broth',
 'broth a',
 'broth-a',
 'brouhaha',
 'Botha',
 'Hasbro',
 'troth',
 'breath',
 'brethren']